In [42]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

PATH = os.getcwd()
PATH

'/home/andre/Documents/Git/SHP_MEC1515'

# Funções

## Compressor

* $Qc$: somatório da vazão demandada por cada elemento
* $K$: Fator que varia entre $1,2$ e $1,5$ (leva em consideração os vazamentos ou eventuais ampliações na planta pnemática)
* $I$: Coeficiente de inserção
    1. $T_t$: Tempo de funcionamento do compressor
    2. $T_s$: Tempo de parada do compressor

In [49]:
class Reservatorio:
    def __init__(self, Tt, Ts, Qc):
        self.Tt = Tt
        self.Ts = Ts
        self.Qc = Qc
        self.I = 0
        self.K = 0

    # Consideracao dos vazemaetos (entre 1.2 e 1.5)
    def set_K(self, K):
        self.K = K

    # I
    def coeficiente_de_insercao(self):
        self.I = (self.Tt / (self.Tt + self.Ts))
        return self.I
    
    def vazao_compressor(self):
        return (self.Qc * self.K / self.I)

In [43]:
def coeficiente_de_insercao(Tt, Ts):
    return (Tt / (Tt + Ts))

def vazao_compressor(Qc, K, I):
    return (Qc * K / I)

## Atuadores (Cilindros)

* $D$: diâmetro do pistão (mm)
* $d$: diâmetro da haste (mm)
* $c$: curso do pistão (mm)
* $n$: número de ciclos por minuto (geralmente por ciclo entende-se uma fase de avanço e recuo)
* $pr$: pressão relativa de regime (MPa - 0,1 MPa = 1 bar)
* $pa$: pressão absoluta de regime (MPa - pa = pr + 0,1)
* $pan$: pressão absoluta de regime da câmara negativa (lado haste)
* $Q$: consumo do ar (litros / min)

In [74]:
class Atuador:
    def __init__(self, D, d, c, pa, n, pressao_igual):
        self.D = D
        self.d = d
        self.c = c
        self.pa = pa
        self.n = n
        self.pressao_igual = pressao_igual
        
        self.consumo = 0
        
    # Consumo de ar de cilindros d0e simples efeito
    def consumo_cilindros_simples(self):
        self.consumo = (self.D**2 * self.c * self.pa * self.n / 127000)
        return self.consumo

    # Consumo de ar de cilindros de duplo efeito
    def consumo_cilindros_duplo_efeito(self):
        if self.pressao_igual == False:
            self.consumo =  (self.c * self.n * (self.D**2 * self.pa + (self.D**2 - self.d**2) \
                * self.pa * self.n / 127000))
            return self.consumo
        
        self.consumo = (self.D**2 * self.c * self.pa * self.n / 63500)
    
        return self.consumo

In [45]:
# Consumo de ar de cilindros d0e simples efeito
def consumo_cilindros_simples(D, c, pa, n):
    return (D**2 * c * pa * n / 127000)

# Consumo de ar de cilindros de duplo efeito
def consumo_cilindros_duplo_efeito(D, d, c, n, pa, pressao_igual):
    if pressao_igual == False:
        return (c * n * (D**2 * pa + (D**2 - d**2) * pa * n / 127000))
    
    return (D**2 * c * pa * n / 63500)

## Reservatório

* $pr$: pressão relativa (MPa)
* $pa_0$: pressão absoluta do ar no estado livre (MPa)
* $V_0$: Volume de ar no estado livre ($m^3$)
* $pa_1$: Pressão absoluta do ar armazenado (MPa)
* $V_1$: Capacidade Total (Vazão total) em $m^3$

In [46]:
def capacidade_reservatorio(pr, V0, pa0=0.1):
    pa1 = pr + pa0
    return (V0 * pa0 / pa1)

# Dados do projeto

In [81]:
# Perimetro total 
perimetro_tubulacao = 2*18 + 7*13 + 2.0

# Quantidade extensoes que irao para 
#os atuadores das portas
extensoes = 20

# Comprimento equivalente dos pontos
# de estrangulamentos
Ts = 11 * 1.7
joelho_90 = 20 * 0.7 + 4 * 0.7
joelho_45 = 20 * 0.4
valvulas_de_pressao = 2.4

Comprimento_total_da_tubulacao = perimetro_tubulacao + Ts + \
                                joelho_45 + joelho_90 + valvulas_de_pressao

print(f'Perimetro total (m): {perimetro_tubulacao}')
print(f'extensoes: {extensoes}')
print(f'Comprimento equivalente dos Ts: {Ts}')
print(f'Comprimento equivalente dos joelhos de 90 graus: {joelho_90}')
print(f'Comprimento equivalente dos joelhos de 45 graus: {joelho_45}')
print(f'Comprimento total: {Comprimento_total_da_tubulacao} m')

Perimetro total (m): 129.0
extensoes: 20
Comprimento equivalente dos Ts: 18.7
Comprimento equivalente dos joelhos de 90 graus: 16.8
Comprimento equivalente dos joelhos de 45 graus: 8.0
Comprimento total: 174.9 m


## Preço Componentes

### Links

1. Pneu_3WaysValve: 
*  [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-3604089439-valvula-esfera-3-vias-l-1-inox-304-bsp-atuador-dupla-aco-_JM) (operation: 0 - 8 bar)

* [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-4657623390-valvula-esfera-3-vias-inox-rosca-34-bsp-atuador-dupla-aco-_JM#polycard_client=recommendations_vip-pads-up&reco_backend=vip-pads-up-experimental-a_marketplace&reco_model=ranker_entity_v2&reco_client=vip-pads-up&reco_item_pos=3&reco_backend_type=low_level&reco_id=c9efef8a-c2c5-4c45-af54-ee44fcfd7bd5&wid=MLB4657623390&sid=recos&is_advertising=true&ad_domain=VIPDESKTOP_UP&ad_position=4&ad_click_id=ZjJiNWZlYzMtYzczNy00YWE1LWJmNzUtN2EyZmU0NjRhM2Vh) (operation: 0 - 8 bar)

* [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-3604089439-valvula-esfera-3-vias-l-1-inox-304-bsp-atuador-dupla-aco-_JM#polycard_client=recommendations_vip-pads-up&reco_backend=vip-pads-up-experimental-a_marketplace&reco_model=ranker_entity_v2&reco_client=vip-pads-up&reco_item_pos=3&reco_backend_type=low_level&reco_id=7a01d7bc-9e61-44f7-9843-dfba6b192f44&wid=MLB3604089439&sid=recos&is_advertising=true&ad_domain=VIPDESKTOP_UP&ad_position=4&ad_click_id=NTMxZjg5ZjEtYTJhYi00NjRhLTliMmQtMmRhN2ZlZWFkZWFl) (operation: 0 - 8 bar)

<br>

2. Pneu_RegisterValve:
* [Pneu_RegisterValve](https://lfautomacao.com.br/products/valvula-esfera-mini-m-f-1-4?variant=45139439026498&country=BR&currency=BRL&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&utm_source=&utm_medium=&utm_campaign=&utm_content=&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEodeQtTI8wxnux3LrEpW_S0SSk5c01HWXtndNtISOookFEkKNXJu4aApdyEALw_wcB) (operation: 0 - 16 bar)

* [Pneu_RegisterValve](https://www.conecfit.com.br/produtos/mini-valvula-esfera-machoxfemea-3-8/?variant=955983600&pf=mc&gad_source=4&gclid=Cj0KCQiA19e8BhCVARIsALpFMgHHpWTi5bUf3u7U7_MfdZOl0nk18PJ7wiccGKI02TRtx7IfDSrl9JQaAgy0EALw_wcB) (operation: 0 - 6 bar)

<br>

3. Pneu_DrainValve:
* [Pneu_DrainValve](https://www.conecfit.com.br/produtos/valvula-solenoide-2vias-16bar-1-2220vca/?variant=956045908&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgH9nbYfXipwIUmtqe3seNuw4dFSEb65lFWCyStoxmobvosGZTppys4aAuIbEALw_wcB) (operation: 0 - 16 bar)

* [Pneu_DrainValve](https://www.tauana.com.br/valvula-solenoide/valvula-solenoide-agua-ar-oleo/vs3513ba04nf-valvula-solenoide-latao-12-bsp?gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEv1Oy0WxygwqY496xu89GEwQgiire1GvkWqmpHUlNtIL_W5flfiG4aAhD-EALw_wcB) (operation: 0 - 8 bar)

* [Pneu_DrainValve](https://www.conecfit.com.br/produtos/valvula-pneumatica-purgadoratimer-220vca/?variant=958126577&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEEGwH-6HRFA7sYwMP67AEbklsD9OuAfP8Mt3r5HZf32A0gOeyOmsgaArQDEALw_wcB) (operation: 0 - 8 bar)

<br>

4. Pneu_Actuator:
* [Pneu_Actuator](https://pt.aliexpress.com/item/4001284845691.html?src=google&pdp_npi=4%40dis!BRL!88.09!88.09!!!!!%40!10000015609702858!ppc!!!&src=google&albch=shopping&acnt=768-202-3196&isdl=y&slnk=&plac=&mtctp=&albbt=Google_7_shopping&aff_platform=google&aff_short_key=UneMJZVf&gclsrc=aw.ds&&albagn=888888&&ds_e_adid=&ds_e_matchtype=&ds_e_device=c&ds_e_network=x&ds_e_product_group_id=&ds_e_product_id=pt4001284845691&ds_e_product_merchant_id=106422254&ds_e_product_country=BR&ds_e_product_language=pt&ds_e_product_channel=online&ds_e_product_store_id=&ds_url_v=2&albcp=21106536414&albag=&isSmbAutoCall=false&needSmbHouyi=false&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgHtk8e-yV2wbOlVEoiwJ6SyA5OHSOXjN4JnHtGSKySfgEn3ivVkSg0aAkS6EALw_wcB) (operation: 0 - 8 bar)

* [Pneu_Actuator](https://uk.c.misumi-ec.com/book/SMC1_ENG_03/digitalcatalog.html?page_num=1) (pages 205 and 251)

* [Pneu_Actuator](https://automationdistribution.com/smc-cm2kf25-200az-cylinder-air/)


<br>

5. Lubrifil
* [Lubrifil](https://www.amazon.com.br/Regulador-Press%C3%A3o-Comprimido-Compressor-Man%C3%B4metro/dp/B0CTYTM6QN/ref=asc_df_B0CTYTM6QN/?tag=googleshopp00-20&linkCode=df0&hvadid=709857067809&hvpos=&hvnetw=g&hvrand=5418949770809360356&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9198996&hvtargid=pla-2292350590546&psc=1&mcid=aae8a2ac30ff3cd08b67a43029c119a2&gad_source=1)

* [Lubrifil](https://www.riberfluid.com.br/mais-produtos/filtro-regulador-dn-14-com-manometro?parceiro=3293&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEwnwHsUgPfqMJXS7Uy8LHoE-dBLVzzeINPP6pFyqYdaVjArhS9bagaAs9IEALw_wcB)

* [Lubrifil](https://www.tauana.com.br/linha-pneumatica/unidade-de-tratamento-de-ar/pn-afc2000-lubrifil-14-com-manometro?gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgFxHPelZIcd6wd7LljXVqhJNLDvNA8Jf2KmiOhYnYAsXhoa80e3ydIaAjMwEALw_wcB)

* [Lubrifil](https://lfautomacao.com.br/products/filtro-regulador-mini-1-4?variant=45161450897730&country=BR&currency=BRL&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&utm_source=&utm_medium=&utm_campaign=&utm_content=&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgG8AIHBAyyvo9oPiAIsXvVjppWWmXl0Ghz9AKZGyGrFMm7X7zEKUesaAtbREALw_wcB)

6. Compressor
* [Compressor](https://www.lojadomecanico.com.br/produto/8480/21/159/Compressor-de-Ar-20-Pes-250-Litros-5HP-220V380V-Trifasico-Alta-Pressao-Industrial-20APV/153/?utm_source=google&utm_medium=cpc&utm_campaign=%5BSOCIAX%5D%5BPMAX%5D%5BROAS%5D+-+COMPRESSORES+E+PNEUM%C3%81TICAS+%5BNOVA%5D&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgFljwDTP6TcADxfohQBAZ56ZNkVPTIwqYwhRMzEC0nsEwvv2O8AfxgaAsVKEALw_wcB) (PCM: 20)

In [89]:
# 20 salas com uma valvula para o acionamento da porta
Pneu_3WaysValve = np.mean([646.95, 710.97, 789.99]) * 22

# 20 salas com 1 registro em cada
Pneu_RegisterValve = np.mean([21.75, 26.26]) * 23

# 2 valvulas de dreno para o sistema, uma qm cada extremidade do predio
Pneu_DrainValve = np.mean([155.97, 209.90, 190.97]) * 2

# 15 janelas e 2 atuadores em cada
Pneu_Actuator = np.mean([119.93, 340.55, 413.97]) * 15 * 2

Lubrifil = np.mean([48.99, 66.54, 90.0, 54.9])

Compressor = np.mean([6199 + 381.23, 6799.90 + 305.96, 4299 + 319.05])

total = Pneu_3WaysValve + Pneu_Actuator + Pneu_DrainValve + \
        Pneu_RegisterValve + Lubrifil + Compressor

print(f'Valvulas direcionais: R$ {Pneu_3WaysValve}')
print(f'Valvulas de Registro: R$ {Pneu_RegisterValve}')
print(f'Valvulas de dreno: R$ {Pneu_DrainValve}')
print(f'Janelas: R$ {Pneu_Actuator}')
print(f'Lubrifil: R$ {Lubrifil}')
print(f'Compressor: R$ {Compressor}')
print(f'Total: R$ {total}')

Valvulas direcionais: R$ 15751.339999999998
Valvulas de Registro: R$ 552.115
Valvulas de dreno: R$ 371.2266666666667
Janelas: R$ 8744.5
Lubrifil: R$ 65.1075
Compressor: R$ 6101.38
Total: R$ 31585.669166666663


In [82]:
atuadores_janelas = Atuador(40, 40, 250, 1, 4, False)
vazao_janelas = atuadores_janelas.consumo_cilindros_simples() * 15 * 2

print(f'Vazao Janelas: {vazao_janelas} L/min')

Vazao Janelas: 377.9527559055118 L/min
